# Game Of Life tutorial in QGIS

Good morning. Today we continue working with rasters and QGIS. We are going to design a spatial algorithm about life and death, also called [Conway's Game of Life](http://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) or cellular automata. 

# Learning outcomes

- Reading and writing rasters with GDAL
- Reading rasters into QGIS via Python console
- Designing spatial algorithm

# Tutorial

Conway’s Game of Life is a raster based zero-player game, where each cell presents an alive or a dead cell. From a certain start situation, each raster-cell can die or come to life depending on their surroundings. Read the [Wikipedia reference]((http://en.wikipedia.org/wiki/Conway%27s_Game_of_Life) carefully to filter the methodology steps needed. Main drivers are as follows:

- Any live cell with fewer than two live neighbours dies, as if caused by under-population.
- Any live cell with two or three live neighbours lives on to the next generation.
- Any live cell with more than three live neighbours dies, as if by overcrowding.
- Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

As a start state the __start_state.tif__ can be used. Let's go have a look what your starting position looks like in QGIS. We will use the start_state as input of our spatial algorithm. At the end of the tutorial the spatial algorithm will output a new end_state. Download the start_state.tif (### add link ###). Open QGIS, open the Python console in QGIS (ctrl + alt + p), set your working directory and load the raster into QGIS.

In [ ]:
import os

# Set working directory and paths
os.chdir("/path/to/workspace")
raster_path = "./input/start_state.tif"

# Visualize Raster into QGIS
startLayer = QgsRasterLayer(raster_path, "start_state")
QgsMapLayerRegistry.instance().addMapLayer(startLayer)

Ok great. The result should look a QR code. You should have a raster of 25 by 25 cells with live cells (white) and dead cells (black). If you feel arty, change the colorscheme into something more lively. Let's check the metadata of the raster file.

In [ ]:
# Import module
from osgeo import gdal, osr

# Load a raster 
openRaster = gdal.Open(raster_path)

# Load the values first band
band = openRaster.GetRasterBand(1)

# Check size of raster
rows = openRaster.RasterYSize
cols = openRaster.RasterXSize

# Check origin and pixel resolution
geotransform = openRaster.GetGeoTransform()
rasterOrigin = (geotransform[0], geotransform[3])
pixelWidth = geotransform[1]
pixelHeight =  geotransform[5]

In the raster metadata information is stored about the origin, pixel resolution and rotation. The origin of a raster is at the top left of all cells. Generally the rotation of maps is towards the North.

- GeoTransform[0] /* top left x */
- GeoTransform[1] /* w-e pixel resolution */
- GeoTransform[2] /* rotation, 0 if image is "north up" */
- GeoTransform[3] /* top left y */
- GeoTransform[4] /* rotation, 0 if image is "north up" */
- GeoTransform[5] /* n-s pixel resolution */

Now read the cell values of the raster into an array by using .ReadAsArray() and specify the cell value type as integers.

In [ ]:
# Import module
import numpy

# Read the first band as an array of integers
array = band.ReadAsArray(0, 0, cols, rows).astype(numpy.int)

print(array)

In the QGIS Python console you should see an array of 25 by 25, which has exactly the same cell values as the raster. 

We have the array loaded, but need to make hard copies of this array to determine the input state and prepare the output state. A soft copy references the array to the input_board/output_board, which keeps a link between the three variables (see [info](https://stackoverflow.com/questions/2612802/how-to-clone-or-copy-a-list?noredirect=1&lq=1). Both input_board and output_board would be linked to the same variable. This would create a wrong output.

Therefore make a hard copy in the following way:

In [ ]:
# Import copy
import copy

# Make a hardcopy of the array
inBoard = copy.copy(array)
sumBoard = copy.copy(array)
outBoard = copy.copy(array)

In [ ]:
# View the cell values of your input board
# You can use the rows and cols variables that you loaded before
# Looping over the rows and columns
for i in range(0, rows): 
    for j in range(0, cols):
        print inBoard[j][i]

Now we need to make an algorithm that would count the neighbours around every cell and store it in an array.
In fact what we need to do is create a window around every raster cell. We can store the sum of neighbours of every raster cell in the sumBoard variable.

In [ ]:
# Looping over the rows and columns
# Then looping over the 8 neighbours around each raster cell
for i in range(0, rows): 
    for j in range(0, cols):
        sumNeighbors = 0 ## counter for neighbours
        boardValue = inBoard[j][i]
        # Count neighbours
        for k in range(i-1, i+2): # look in cells to top and bottom
            for l in range(j-1, j+2):# look in cells to left and right
                ## Only count neighbours and not the cell itself
                if (l,k) != (j,i):
                    sumNeighbors = sumNeighbors+inBoard[l%cols][k%rows]
        sumBoard[i][j] = sumNeighbors
        
print sumBoard

Now we know the amount of neighbours for each cell. But do you know how the amount of neighbours is calculated on the outer edge?

> __Question 1:__ How are the amount of neighbours calculated for edge cases?

Remember the rules from before? Based on __sumBoard__ we can determine which cells should live and which should die. Implement the rules yourself based on the amount of neighbours you counted.

- Any live cell with fewer than two live neighbours dies, as if caused by under-population.
- Any live cell with two or three live neighbours lives on to the next generation.
- Any live cell with more than three live neighbours dies, as if by overcrowding.
- Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

We can create the output board based on the sumboard. Or perhaps an even better solution would be to write directly to the output board.

In [ ]:
# Looping over the rows and columns
# Then looping over the 8 neighbours around each raster cell
# After counting neighbours for cell, decide if that cell should be dead or alive
for i in range(0, rows): 
    for j in range(0, cols):
        sumNeighbors = 0 ## counter for neighbours
        boardValue = inBoard[j][i]
        #Count neighbours
        for k in range(i-1, i+2): 
            for l in range(j-1, j+2):
                ## Only count neighbours and not the cell itself
                if (l,k) != (j,i):
                    sumNeighbors = sumNeighbors+inBoard[l%cols][k%rows]	
        #Alive cells
        if boardValue == 1:
            ## Under-population
            if sumNeighbors < 2:
                outBoard[j][i] = 0
            ## Overcrowding
            elif sumNeighbors > 3:
                outBoard[j][i] = 0
        #Dead cells -> Reproduction
        elif boardValue == 0 and sumNeighbors == 3:
            outBoard[j][i] = 1
            
            
print outBoard

We are working with four loops to look for every cell (with X and Y position) what its neighbouring cells (with X and Y position) are. 

> __Question 2:__ How many loops would you need to update the states in a Game of Live 3D version?

> __Question 3:__ Are we sequentially processing the cell values or are we processing cell values parellized? Which would be more efficient?

We can save the new state that we created as a raster. Let's define a path and save the array as a raster.

In [ ]:
# set output path
output_path = "./output/end_state.tif"

# Load driver
driver = gdal.GetDriverByName('GTiff')

# Create empty raster file with integer type
outRaster = driver.Create(output_path,cols,rows,1,gdal.GDT_Int32)

# Set the metadata for geotransformation. 
outRaster.SetGeoTransform(geotransform)

# Create band which will hold cell values
outband = outRaster.GetRasterBand(1)
outband.WriteArray(outBoard,0,0)

# Set the correct coordinate system
outRasterSRS = osr.SpatialReference()
outRasterSRS.ImportFromEPSG(4326)
outRaster.SetProjection(outRasterSRS.ExportToWkt())

# Flush outband and outRaster to save the raster
outband.FlushCache()
outRaster.FlushCache()

By adding the __end_state.tif__ to QGIS we can see the updated state.
We learned how to read and write rasters and how to create a spatial algorithm.

# Exercise lesson 13

In the tutorial the state was only changed once. But the Game of Life gets more interesting if we change the state multiple times. Life is not a moment, but a lifetime of fun.

The same input raster from the tutorial is used. However this time  change the Game of Life into a function that changes the state iteratively over many cycles.

Make sub functions:
- ReadRasterToArray
- UpdateGameOfLife
- WriteArrayToRaster

Make a main function GameOfLife() that uses the above functions and has three input arguments (input_path, output_path, cycle_number).

Bonus: If you add code to find how many cycles are needed to get a stable state from the starting state. The stable state is reached before 2000 cycles.

Hint: You can loop inside the UpdateGameOfLife function or you can setup a loop in the main function. 

Use the Python editor in QGIS to write and run your Python script. Don't forget to work with a project structure and to hand in your script!

Extra, not bonus: If you want to do something more elaborate, try to make a standalone Python script outside of QGIS in Spyder that uses some functionality of QGIS by importing QGIS. If Spyder is not installed, type "sudo apt-get install spyder". If QGIS is installed, you can add functionality of QGIS to standalone Python scripts by adding the two lines below.

In [ ]:
import qgis, numpy, os, time, shutil
from qgis.core import QgsMapLayerRegistry, QgsRasterLayer